In [26]:
#IMPORTANDO BIBLIOTECAS NECESSÁRIAS
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

In [27]:
#ABRINDO ARQUIVO EM DATAFRAME
df = pd.read_csv('dados_simulacao2.csv')

In [28]:
#VISUALIZAÇÃO
df.head(5)

,Nome,LOA (m),Boca (m),DWT (ton),Calado de Entrada (m),Calado de Saída (m),Calado Aéreo (m),Pontal (m),Tamanho de Lança (m),Ano de Construção,Tipo do Navio,Último Porto,Próximo Porto,Situação
0,Navio1,59,15,63260,14,8,24,19,11,2008,GL,Porto5,Porto2,1
1,Navio2,15,9,46723,13,15,30,13,60,2015,GL,Porto1,Porto5,1
2,Navio3,18,11,191817,15,16,28,10,33,2009,GS,Porto3,Porto1,0
3,Navio4,18,13,195241,11,16,19,12,74,2020,GSM,Porto2,Porto5,0
4,Navio5,54,17,37299,10,15,16,14,24,2006,GLP,Porto5,Porto3,1


In [29]:
#INFOS SOBRE TIPAGEM DE DADOS DO DF
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Nome                   1000 non-null   object
 1   LOA (m)                1000 non-null   int64 
 2   Boca (m)               1000 non-null   int64 
 3   DWT (ton)              1000 non-null   int64 
 4   Calado de Entrada (m)  1000 non-null   int64 
 5   Calado de Saída (m)    1000 non-null   int64 
 6   Calado Aéreo (m)       1000 non-null   int64 
 7   Pontal (m)             1000 non-null   int64 
 8   Tamanho de Lança (m)   1000 non-null   int64 
 9   Ano de Construção      1000 non-null   int64 
 10  Tipo do Navio          1000 non-null   object
 11  Último Porto           1000 non-null   object
 12  Próximo Porto          1000 non-null   object
 13  Situação               1000 non-null   int64 
dtypes: int64(10), object(4)
memory usage: 109.5+ KB


In [30]:
df.describe()

,LOA (m),Boca (m),DWT (ton),Calado de Entrada (m),Calado de Saída (m),Calado Aéreo (m),Pontal (m),Tamanho de Lança (m),Ano de Construção,Situação
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,36.611000,11.425000,113554.570000,13.536000,13.567000,21.698000,12.153000,63.278000,2009.929000,0.408000
std,12.956371,3.527722,50034.347423,3.408272,3.459694,7.342118,4.443798,31.249564,7.156478,0.491709
min,15.000000,6.000000,30043.000000,8.000000,8.000000,10.000000,5.000000,10.000000,1998.000000,0.000000
25%,25.000000,8.000000,69858.750000,11.000000,11.000000,15.000000,8.000000,37.000000,2003.750000,0.000000
50%,37.000000,11.000000,109808.500000,14.000000,14.000000,22.000000,12.000000,61.500000,2010.000000,0.000000
75%,48.000000,14.000000,158915.250000,16.000000,17.000000,28.000000,16.000000,90.000000,2016.000000,1.000000
max,59.000000,17.000000,199967.000000,19.000000,19.000000,34.000000,19.000000,119.000000,2022.000000,1.000000


In [31]:
#BALANCEAMENTO DE CLASSE
df['Situação'].nunique

<bound method IndexOpsMixin.nunique of 0      1
1      1
2      0
3      0
4      1
      ..
995    0
996    0
997    0
998    1
999    1
Name: Situação, Length: 1000, dtype: int64>

In [32]:
#DEFINIÇÃO DOS PREVISORES E CLASSE
indices_previsores = [1, 2, 3, 4, 5, 9]
previsores = df.iloc[:, indices_previsores].values
classe = df.iloc[:, 13].values

In [33]:
#SEPARANDO DADOS DE TESTE E TREINO
p_train, p_test, c_train, c_test = train_test_split(previsores, classe, test_size=0.2, random_state=0)


In [34]:
# DEFININDO PARÂMETROS PARA RANDOM FOREST
parametros = {
    'n_estimators':[10, 20, 30],
    'max_depth':[None, 10, 20],
    'min_samples_split':[2,5,10],
    'min_samples_leaf':[1,2,4],
    'max_features':['sqrt', 'log2'],
    'criterion':['gini', 'entropy'],
    'bootstrap':[True, False]
}

In [35]:
# CRIAÇÃO DO CLASSIFICADOR BASE
classificador_base = RandomForestClassifier(random_state=0)

In [36]:
# CONFIGURANDO GRIDSEARCHCV PARA MELHOR ESCOLHA DOS HIPERPARÂMETROS
grid_search = GridSearchCV(classificador_base, parametros, cv=5, scoring='accuracy')

In [37]:
# EXECUTANDO A BUSCA EXAUSTIVA
grid_search.fit(p_train, c_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [10, 20, 30]},
             scoring='accuracy')

In [38]:
# OBTENDO MELHORES PARÂMETROS
melhores_parametros = grid_search.best_params_
melhores_parametros

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 10}

In [39]:
# AJUSTANDO NO CLASSIFICADOR
classificador = RandomForestClassifier(
    bootstrap=True,
    criterion='entropy',
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=30
)
classificador

RandomForestClassifier(criterion='entropy', n_estimators=30)

In [40]:
classificador.fit(p_train, c_train)

RandomForestClassifier(criterion='entropy', n_estimators=30)

In [41]:
previsoes = classificador.predict(p_test)

In [42]:
precisao = accuracy_score(c_test, previsoes)
matriz = confusion_matrix(c_test, previsoes)

print("RESULTADO DE CONFUSION MATRIZ:\n", matriz)
print("\nACURÁCIA:",precisao)

RESULTADO DE CONFUSION MATRIZ:
 [[130   0]
 [  0  70]]

ACURÁCIA: 1.0


In [43]:
resultados_acuracia = cross_val_score(classificador, previsores, classe, cv=5, scoring='accuracy')

print("Resultados Acurácia: ", resultados_acuracia)
print("Acurácia média: ", np.mean(resultados_acuracia))

Resultados Acurácia:  [1. 1. 1. 1. 1.]
Acurácia média:  1.0
